## 0. Cargando la Data y Packages

In [14]:
using Pkg
Pkg.add("CSV")
Pkg.add("RData")
Pkg.add("DataFrames")
Pkg.add("StatsBase")
Pkg.add("StatsModels")
Pkg.add("Statistics")
Pkg.add("GLM")
Pkg.add("LaTeXStrings")
Pkg.add("CategoricalArrays")


   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed Requires ────────── v1.3.0
   Installed Scratch ─────────── v1.2.1
   Installed FileIO ──────────── v1.16.3
   Installed Mocking ─────────── v0.8.1
   Installed TZJData ─────────── v1.2.0+2024a
   Installed RData ───────────── v1.0.0
   Installed TimeZones ───────── v1.18.0
   Installed ExprTools ───────── v0.1.10
   Installed CategoricalArrays ─ v0.10.8
    Updating `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  [df47a6cb] + RData v1.0.0
    Updating `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
  [324d7699] + CategoricalArrays v0.10.8
  [e2ba6199] + ExprTools v0.1.10
  [5789e2e9] + FileIO v1.16.3
  [78c3b35d] + Mocking v0.8.1
  [df47a6cb] + RData v1.0.0
  [ae029012] + Requires v1.3.0
  [6c6a2e73] + Scratch v1.2.1
  [dc5dba14] + TZJData v1.2.0

In [21]:
using CSV
using GLM
using DataFrames
using LaTeXStrings
using StatsBase
using Statistics,RData 
using CategoricalArrays

#podemos cargar esto como csv
df_csv = CSV.read("../../../data/wage2015_subsample_inference.csv", DataFrame)
#pero también como RData
df_rdata = load("../../../data/wage2015_subsample_inference.RData")["data"]

#en este caso usaremos el archivo RData sobretodo porque detecta Categoricals
df = df_rdata
#para ver las primeras 5 observacione

first(df, 5) 


Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600,11,8370,18
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050,10,5070,9
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260,19,770,4
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420,1,6990,12
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015,6,9470,22


## 1. Data analysis

#### 2. Are there missing values? Display the number of missings (if any) of each variable in the data set.


#### 3. Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [23]:
# Usamos describe para obtener algunos estadísticos descriptivos y ver los missingvalues de caca variable del data set
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,wage,23.4104,3.02198,19.2308,528.846,0,Float64
2,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
3,sex,0.444466,0.0,0.0,1.0,0,Float64
4,shs,0.023301,0.0,0.0,1.0,0,Float64
5,hsg,0.243883,0.0,0.0,1.0,0,Float64
6,scl,0.278058,0.0,0.0,1.0,0,Float64
7,clg,0.31767,0.0,0.0,1.0,0,Float64
8,ad,0.137087,0.0,0.0,1.0,0,Float64
9,mw,0.259612,0.0,0.0,1.0,0,Float64


In [24]:
using Statistics
#encontramos algunos los cuartiles de la variable wage:
q1 = quantile(df.wage, 0.25)  
q2 = quantile(df.wage, 0.50)  
q3 = quantile(df.wage, 0.75)  

#imprimimos los valores de cada cuartil
println("Q1: $q1")
println("Mediana: $q2")
println("Q3: $q3")


Q1: 13.461538461538462
Mediana: 19.23076923076923
Q3: 27.77777777777778


### 4. How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result

In [25]:
#nos están preguntando por cuantas muejeres graduadas de universidad o con postgrado se encuentran el cuartil superior de salarios. (mayores a 27.7777)

df[(df.sex .== 1.0) .& ((df.clg .== 1.0) .|| (df.ad .== 1.0)) .& (df.wage .>= q3), :]

Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…
1,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015,6,9470,22
2,42.3077,3.74497,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4700,16,4795,9
3,41.2088,3.71865,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,11.0,1.21,1.331,1.4641,4700,16,5790,9
4,100.0,4.60517,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,27.0,7.29,19.683,53.1441,850,2,6970,12
5,33.6538,3.51613,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.16,0.064,0.0256,710,2,7390,14
6,36.0577,3.58512,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,23.0,5.29,12.167,27.9841,420,1,9170,21
7,31.25,3.44202,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,15.0,2.25,3.375,5.0625,3255,10,8190,18
8,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9.0,0.81,0.729,0.6561,540,2,6990,12
9,48.0769,3.8728,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,24.0,5.76,13.824,33.1776,4850,16,3390,6


In [26]:
#para saber el número de mujeres que cumplen con estos requisitos:
mujeres_with_college_or_superior_degree_in_q3=nrow(df[(df.sex .== 1.0) .& ((df.clg .== 1.0) .|| (df.ad .== 1.0)) .& (df.wage .>= q3), :])

println("Las mujeres con universidad o un grado superior en el cuartil superior de salarios son: $mujeres_with_college_or_superior_degree_in_q3")

Las mujeres con universidad o un grado superior en el cuartil superior de salarios son: 419


#### 5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [27]:
#hacemos algo similar para los hombres:
men_with_highschool_or_no_degree_in_q3=nrow(df[(df.sex .== 0.0) .& ((df.shs .== 1.0) .|| (df.hsg .== 1.0)) .& (df.wage .>= q3), :])

println("Los hombres con high school terminada o un grado menor en el cuartil superior de salarios son: $men_with_highschool_or_no_degree_in_q3")

Los hombres con high school terminada o un grado menor en el cuartil superior de salarios son: 118


#### 6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [28]:
#creamos los 2 dataframes

#1er dataframe (only log(wage))

only_logwage = DataFrame(log_wage = df.lwage)

#2do dataframe (otras variables)

df_no_wage = select(df, Not([:wage, :lwage]))



Row,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600,11,8370,18
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050,10,5070,9
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260,19,770,4
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420,1,6990,12
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015,6,9470,22
6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650,5,7460,14
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120,17,7280,14
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240,17,5680,9
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040,13,8590,19


## 2. Data wrangling

#### 7. Make an array for our Y variable, which will be the logarithm of wage (lwage column)

In [29]:
#convierto en array con nombre Y
Y=df.lwage

5150-element Vector{Float64}:
 2.2633643798407643
 3.872802292274865
 2.403126322215923
 2.634927936273247
 3.361976668508874
 2.4622152385859297
 2.9565115604007097
 2.9565115604007097
 2.4849066497880004
 2.9565115604007097
 ⋮
 3.117779707996832
 2.822980167776187
 3.1796551117149194
 2.6280074934286737
 2.6925460145662448
 3.138833117194664
 3.649658740960655
 3.4955080611333966
 2.8511510447428834

In [31]:
#primero convertimos en matriz  y quitamos las variables que no vamos a usar:
no_wage = Matrix(select(df, Not([:wage, :lwage, :occ, :ind],)))


5150×16 Matrix{Any}:
 1.0  0.0  0.0  0.0  1.0  0.0  0.0  …   0.49   0.343    0.2401  "11"  "18"
 0.0  0.0  0.0  0.0  1.0  0.0  0.0      9.61  29.791   92.3521  "10"  "9"
 0.0  0.0  1.0  0.0  0.0  0.0  0.0      3.24   5.832   10.4976  "19"  "4"
 1.0  0.0  0.0  0.0  0.0  1.0  0.0      6.25  15.625   39.0625  "1"   "12"
 1.0  0.0  0.0  0.0  1.0  0.0  0.0      4.84  10.648   23.4256  "6"   "22"
 1.0  0.0  0.0  0.0  1.0  0.0  0.0  …   0.01   0.001    0.0001  "5"   "14"
 1.0  0.0  1.0  0.0  0.0  0.0  0.0     17.64  74.088  311.17    "17"  "14"
 0.0  0.0  1.0  0.0  0.0  0.0  0.0     13.69  50.653  187.416   "17"  "9"
 1.0  0.0  1.0  0.0  0.0  0.0  0.0      9.61  29.791   92.3521  "13"  "19"
 1.0  0.0  0.0  0.0  1.0  0.0  0.0      0.16   0.064    0.0256  "10"  "18"
 ⋮                        ⋮         ⋱                                 ⋮
 0.0  0.0  1.0  0.0  0.0  0.0  0.0      2.25   3.375    5.0625  "22"  "9"
 0.0  0.0  1.0  0.0  0.0  0.0  0.0      1.21   1.331    1.4641  "20"  "21"
 1.0  0.0  

In [33]:
#filtramos y nos quedamos con las variables que nos contengan wage y que nos interesen
df_no_wage = select(df, Not([:wage, :lwage, :rownames, :occ, :ind]))

first(df_no_wage, 5)

Row,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ2,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,11,18
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,10,9
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,19,4
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,1,12
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,6,22


In [45]:
#vemos los valores que toma la variable occ2 para luego convertirla en dummie
sort(unique(df_no_wage.occ2))
#efectivamente son valores del 1 al 22. Por tanto podemos crear una dummie tomando como base a 1

22-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 14
 15
 16
 17
 18
 19
 20
 21
 22

In [32]:
#vemos los valores que toma la variable ind2 para luego convertirla en dummie
sort(unique(df_no_wage.ind2))
#efectivamente son valores del 2 al 22. Por tanto podemos crear una dummie tomando como base a

21-element Vector{String}:
 "10"
 "11"
 "12"
 "13"
 "14"
 "15"
 "16"
 "17"
 "18"
 "19"
 ⋮
 "21"
 "22"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 "9"

In [40]:
#eliminando variables que pueden causar multicolinealidad
df = dropmissing(df, [:ad, :ne])

Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Cat…,Cat…,Cat…
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600,11,8370,18
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050,10,5070,9
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260,19,770,4
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420,1,6990,12
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015,6,9470,22
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650,5,7460,14
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120,17,7280,14
8,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240,17,5680,9
9,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040,13,8590,19


In [41]:
basic_model = lm(@formula(lwage ~ sex+shs+hsg+scl+clg+mw+so+we+occ2+ind2), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + shs + hsg + scl + clg + mw + so + we + occ2 + ind2

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)   Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)   3.83899     0.0811716   47.29    <1e-99   3.67986     3.99812
sex          -0.0649486   0.0152641   -4.25    <1e-04  -0.0948727  -0.0350244
shs          -0.53551     0.0512017  -10.46    <1e-24  -0.635887   -0.435132
hsg          -0.459379    0.0273044  -16.82    <1e-61  -0.512907   -0.40585
scl          -0.386357    0.0255448  -15.12    <1e-49  -0.436435   -0.336278
clg          -0.187943    0.0233293   -8.06    <1e-15  -0.233678   -0.142207
mw           -0.038303

## 3. Linear Regressions